In [65]:
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import base64
import time
from bs4 import BeautifulSoup
from urllib.parse import quote

In [78]:
ano = 2011

df_edicao = pd.read_csv(f'./concept_drift/tabela_final_{ano}.csv')
codigo_cursos = df_edicao["Codigo_do_Curso"].unique().tolist()

dict_codigo_ce = {
    codigo: 0
    for codigo
    in codigo_cursos
}

In [79]:
async def fetch(session, codigo, timeout):
    utf8_codigo_curso = str(codigo).encode("utf-8")
    base64_codigo_curso = base64.b64encode(utf8_codigo_curso).decode("utf-8")
    base64_utf8_codigo_curso = quote(base64_codigo_curso, encoding="utf-8")
    #print(base64_utf8_codigo_curso)
        
    async with session.get(f'https://emec.mec.gov.br/emec/consulta-curso/listar-historico-indicadores-curso/c1999930082674af6577f0c513f05a96/{base64_utf8_codigo_curso}/list/10', 
                           timeout=timeout) as response:
        html_table = await response.text()
        soup = BeautifulSoup(html_table, 'html.parser')

        rows = soup.find('tbody').find_all('tr')

        row_edicao = None
        for row in rows:
            if row.find('td', text=f'{ano}'):
                row_edicao = row
                break

        if row_edicao:
            enade_edicao = row_edicao.find_all('td')[1].text
            dict_codigo_ce[codigo] = enade_edicao

async def fetch_all():
    result = []
    timeout = aiohttp.ClientTimeout(total=150)
    conn = aiohttp.TCPConnector(limit=10, limit_per_host=10)
    
    #codigo_cursos = lista_codigo_curso[1384:1605]
    #codigo_cursos = [73438, 1166493, 1169549, 1170053, 1170480, 1174506, 1179304, 1180251, 1180846]

    async with aiohttp.ClientSession(timeout=timeout, connector=conn) as session:
        await asyncio.gather(*[fetch(session, codigo, timeout) for codigo in codigo_cursos], return_exceptions=True)
        #return result

In [80]:
await fetch_all()

In [81]:
len(dict_codigo_ce)

1222

In [82]:
print(dict_codigo_ce)

{36: '3', 127: '4', 317: '3', 513: '3', 594: '3', 614: '3', 631: '3', 703: '5', 778: '3', 865: '3', 968: '3', 1110: '4', 1261: '4', 1381: '3', 1452: '4', 1529: '4', 1532: '3', 1633: '3', 1721: '3', 1904: '4', 2090: '3', 2209: '4', 2310: '3', 2429: '4', 2712: '3', 2717: '3', 3190: '3', 3193: '5', 3197: '3', 3211: '3', 3419: '3', 3531: '2', 3676: '3', 3762: '1', 3858: '2', 3859: '3', 3935: '3', 3952: '1', 4547: '3', 4675: '2', 4963: '2', 5079: '2', 5123: '2', 5238: '2', 5390: '3', 5432: '3', 5485: '3', 5841: '2', 5913: '2', 6131: '1', 6491: '5', 6582: '2', 6668: '2', 6756: '3', 6847: '2', 6852: '2', 6858: '3', 7142: '2', 7292: '4', 7301: '3', 7511: '4', 7523: '3', 7839: '4', 8170: '3', 8340: '2', 8684: '2', 8913: '2', 9302: '2', 9311: '3', 9414: '3', 9425: '3', 9440: '4', 9470: '2', 9543: '2', 9586: '2', 10005: '4', 10059: '3', 10077: '3', 10082: '2', 10159: '2', 10348: '2', 10515: '2', 10728: '3', 10744: '2', 10814: '3', 10978: '2', 10999: '2', 11238: '3', 11452: '4', 11554: '2', 11730:

In [83]:
lista_ce = list(dict_codigo_ce.values())
lista_codigo_curso = list(dict_codigo_ce.keys())

In [ ]:
lista_codigo_curso

In [84]:
data = {
    "Codigo_Curso": lista_codigo_curso,
    "Nota_Conceito_Faixa": lista_ce
}

In [85]:
df_ce_edicao = pd.DataFrame(data)
df_ce_edicao = df_ce_edicao.astype({'Nota_Conceito_Faixa':'int64'})

In [95]:
df_ce_edicao

,Codigo_Curso,Nota_Conceito_Faixa
0,36,3
1,127,4
2,317,3
3,513,3
4,594,3
...,...,...
1216,5000356,3
1217,5000448,2
1218,5000456,3
1219,5000457,5


In [96]:
df_ce_edicao.to_csv(f'./conceito_enade_{ano}_webscrap.csv',index=False)

###########################################
#### Para testes dos resultados

In [88]:
utf8_codigo_curso = str(94260).encode("utf-8")
base64_codigo_curso = base64.b64encode(utf8_codigo_curso).decode("utf-8")
base64_utf8_codigo_curso = quote(base64_codigo_curso, encoding="utf-8")
base64_utf8_codigo_curso

'OTQyNjA%3D'

In [97]:
df_ce_edicao[df_ce_edicao["Nota_Conceito_Faixa"] == 0]["Codigo_Curso"].to_list()

[]

In [93]:
# Edição de 2017
# df_ce_edicao = df_ce_edicao[df_ce_edicao["Codigo_Curso"] != 73438]

# Edição de 2011
# df_ce_edicao = df_ce_edicao[df_ce_edicao["Codigo_Curso"] != 94260]

# df_ce_edicao.sort_values("Codigo_Curso", inplace = True)
# df_ce_edicao.reset_index(drop=True, inplace = True)

/tmp/ipykernel_58934/2099259377.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ce_edicao.sort_values("Codigo_Curso", inplace = True)
